### This is a sample code running on a sample AccountOID to get the forecasted water consumption for 30 days.

#### Note: Provided a sample database that contains the following attributes: AccountOID: Account ID, ConsumptionDate: Consumption Date, Consumption: Water daily Consumption (m^3)
#### Note: The following requirements were used pandas==1.4.3, pmdarima==2.0.1, statsmodels==0.13.2, pyodbc==4.0.34




In [ ]:
import pandas as pd
from pmdarima.arima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
import pyodbc

warnings.filterwarnings("ignore")

In [ ]:
##Pyodbc connection 
try:
    cnxn = pyodbc.connect(driver='{ODBC Driver 17 for SQL Server}',
                        server='******',
                        database='******',
                        uid='******',
                        pwd='******')
    cursor = cnxn.cursor()
    
except pyodbc.Error as ex:
    sqlstate = ex.args[1]
    sqlstate = sqlstate.split(".")
    print(sqlstate)
        
sqlfilter = """
             SELECT
             [ConsumptionDaily].[ConsumptionDate],
             [ConsumptionDaily].[Consumption]
            FROM
             [PORTAL].[ConsumptionDaily]
            WHERE [ConsumptionDaily].[AccountOID] = {var1}
            ORDER BY
             [ConsumptionDaily].[ConsumptionDate]
             ;
            """.format(var1=3225298)  #sample 
df_main = pd.read_sql_query(sqlfilter, cnxn)

## Data Processing

In [ ]:
def process(df):
    
    df = df.drop('AccountOID', axis=1)
    dates = df['ConsumptionDate'].to_list()
    all_dates = pd.date_range(dates[0], dates[-1])
    all_dates_df = pd.DataFrame(pd.date_range(dates[0], dates[-1]), columns=['ConsumptionDate'])
    df['ConsumptionDate'] = pd.to_datetime(df['ConsumptionDate'])
    df = pd.merge(all_dates_df, df, on='ConsumptionDate', how='left')
    df['Consumption'] = df['Consumption'].fillna(0)
    df["ConsumptionDate"] = pd.to_datetime(df["ConsumptionDate"])
    df.loc[df['Consumption'] > 800, 'Consumption'] = df['Consumption'].median()
    df = df.set_index('ConsumptionDate')
    
    return df

data_processed = process(df_main.copy())
data_processed

## Modeling

In [ ]:
def modeling(dataset):

    data_size = dataset.shape[0]
    test_size = 30
    train_X, train_y = dataset[:data_size - test_size], dataset[:data_size - test_size]
    test_X, test_y = dataset[data_size - test_size:], dataset[data_size - test_size:]

    #Automating ARIMA to choose best parameters    
    step_wise = auto_arima(train_y,exogenous= train_X, seasonal=True,start_p=1, start_q=1, max_p=7, max_q=7,
    error_action = 'ignore', suppress_warnings = True, stepwise=True, parallel = True, verbose=False)

    #SARIMAX modeling with the choosen parameters  
    model= SARIMAX(train_y,exog=train_X,order=step_wise.order)
    results = model.fit()
    #Predictions
    predictions = results.predict(start=data_size - test_size, end=data_size - 1, exog=np.array(test_X))

    return predictions

In [ ]:
res = modeling(data_processed)